In [1]:
from sklearn.linear_model import LogisticRegression

from cuisine.cookbook import get_ingredient_list, get_cookbook_train, get_cookbook_valid_question, get_cookbook_valid_answer, get_cookbook_test_question
from cuisine.embedding import import_embedding, create_random_embedding, create_one_hot_embedding
from cuisine.chef import make_recipe_embedding_data
from cuisine.utils import move_to_top_directory

In [2]:
move_to_top_directory()
%pwd

'/home/felix/cuisine'

In [3]:
cookbook_train = get_cookbook_train()
cookbook_valid_question = get_cookbook_valid_question()
cookbook_valid_answer = get_cookbook_valid_answer()
cookbook_test_question = get_cookbook_test_question()
ingredient_list = get_ingredient_list()

cookbook_train[15]

{'recipe_id': 15,
 'ingredients': [2866, 4243, 4362, 5377, 5408, 6187, 6352, 6568],
 'kitchen_name': 'italian',
 'kitchen_id': 5}

In [4]:
embedding = create_one_hot_embedding(ingredient_list)

X_train, y_train = make_recipe_embedding_data(cookbook_train, embedding, avg=False)
X_valid, _       = make_recipe_embedding_data(cookbook_valid_question, embedding, avg=False)
_      , y_valid = make_recipe_embedding_data(cookbook_valid_answer, embedding, avg=False)
X_test , _       = make_recipe_embedding_data(cookbook_test_question, embedding, avg=False)

clf = LogisticRegression(penalty='l2', max_iter=100).fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
valid_score = clf.score(X_valid, y_valid)
print(f"Training score {train_score:.3f} and validation score {valid_score:.3f}")
prediction = clf.predict(X_test).tolist()

/home/felix/anaconda3/envs/cuisine/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training score 0.879 and validation score 0.761


In [10]:
import csv

with open('data/test_classification_answer.csv', 'w') as test_answer:

    writer = csv.writer(test_answer)

    for kitchen_id in prediction:
        for recipe in cookbook_train:
            if recipe['kitchen_id'] == kitchen_id:
                writer.writerow([recipe['kitchen_name']])
                break